#  Setup stuff: Connectivity

In [ ]:
#  From,
#
#     https://github.com/KatanaGraph/katana-enterprise/blob/master/python/test/notebooks/red/analytics_remote.ipynb


In [ ]:
import os

from katana import remote
from katana.remote import import_data


os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

In [ ]:
DB_NAME    = "my_db"
GRAPH_NAME = "my_graph"

print("--")

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Setup stuff: Debugging 

In [ ]:
#  LIST ALL GRAPHS

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
my_client.widgets().operations()


In [ ]:
# ##################################################################

In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


#  Actual analytics ..

In [ ]:

import numpy as np
from katana import remote
from katana.remote import analytics
from katana_enterprise.distributed import PartitionPolicy


l_cntr = 0

print("--")


In [ ]:

l_source_nodeid       = 2                          #  id of node to start on (this is SJC)

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.bfs(
   my_graph,
   source               = l_source_nodeid,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

display(print("--"))


In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
l_result.view()


In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
l_result.view()


#  Graph loaded, but before running sssp() 
#     (This is correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge
#  3            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge
#
#  1            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge

#  Graph loaded, after running sssp() 
#     (This is not correct)
#
#  r.end.id     r.end.labels    r.internal_id   r.label     r.DISTANCE  r.NUM_HOPS  r.start.id  r.start.labels  r.type
#  1            ['Airport']     0               FLIES_TO      66        1           0           ['Airport']     edge
#  0            ['Airport']     1               FLIES_TO      66        1           1           ['Airport']     edge
#
#  3            ['Airport']     2               FLIES_TO     886        1           1           ['Airport']     edge
#  1            ['Airport']     5               FLIES_TO     886        1           3           ['Airport']     edge
#
#  2            ['Airport']     6               FLIES_TO     948        1           3           ['Airport']     edge         #  This data is now wrong
#  1            ['Airport']     3               FLIES_TO     948        1           2           ['Airport']     edge         #  This data is now wrong
#
#  3            ['Airport']     4               FLIES_TO    1829        1           2           ['Airport']     edge         #  This data is now wrong



In [ ]:
l_result = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN n, r, m
   """ ,
   contextualize=True,
   )
l_result.view()
